In [1]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
Tesla T4


In [2]:
import os
import torch

os.environ["OMP_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
torch.set_num_threads(2)

In [4]:
from relik import Relik
from relik.inference.data.objects import RelikOutput

# Load model on GPU, index on CPU
relik = Relik.from_pretrained(
    "relik-ie/relik-cie-small",
    device="cuda",  # model runs on GPU
    index_device="cpu",  # document embeddings stay on CPU
    precision="fp16",  # optional: reduce model memory further
    topk=50,
)

text = """UPS plane crashes after takeoff from Louisville airport. A fireball erupts near airport property after reports of a plane crash at Louisville International Airport, Tuesday, Nov. 4, 2025, in Louisville, Ky.
A UPS cargo plane crashed shortly after takeoff from Louisville Muhammad Ali International Airport on Tuesday evening, federal officials said.
According to the Federal Aviation Administration, UPS Flight 2976 went down around 5:15 p.m. local time after departing the airport. The McDonnell Douglas MD-11 was headed to Daniel K. Inouye International Airport in Honolulu.
Flames and heavy smoke were seen rising from the airfield as fire crews worked to extinguish the blaze.
It’s not yet known whether there were any injuries or fatalities. The cause of the crash remains under investigation.
The FAA said both the agency and the National Transportation Safety Board will investigate."""

# Run inference
out: RelikOutput = relik(text)

                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2025-12-13 15:28:38,574] [INFO] [relik.inference.annotator.from_pretrained:700] [PID:77365] [RANK:0] Loading Relik from relik-ie/relik-cie-small
[2025-12-13 15:28:38,576] [INFO] [relik.inference.annotator.from_pretrained:701] [PID:77365] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'span': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'relik-ie/index-e5-base-v2-wikipedia-matryoshka',
        },
        'triplet': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'relik-ie/encoder-e5-small-v2-wikipedia-relations-index',
        },
    },
    'metadata_fields': [],
    'reader': {
        '_target_': 'relik.reader.pytorch_modules.triplet.RelikReaderForTripletExtraction',
        'transformer_model': 'relik-ie/relik-reader-deberta-v3-small-cie-wikipedia',
        'use_nme': True,
    },
  

In [7]:
high_conf_triples = [t for t in out.triplets if t.confidence >= 0.50]

print(high_conf_triples)

[Triplets(subject=Span(start=0, end=3, label='UPS Airlines', text='UPS'), label='airline hub', object=Span(start=37, end=55, label='Louisville International Airport', text='Louisville airport'), confidence=0.8999999761581421), Triplets(subject=Span(start=209, end=212, label='UPS Airlines', text='UPS'), label='airline hub', object=Span(start=260, end=305, label='Louisville International Airport', text='Louisville Muhammad Ali International Airport'), confidence=0.7599999904632568), Triplets(subject=Span(start=400, end=415, label='--NME--', text='UPS Flight 2976'), label='start point', object=Span(start=470, end=481, label='Cleveland Hopkins International Airport', text='the airport'), confidence=0.6200000047683716), Triplets(subject=Span(start=525, end=563, label='Daniel K. Inouye International Airport', text='Daniel K. Inouye International Airport'), label='location', object=Span(start=567, end=575, label='Honolulu', text='Honolulu'), confidence=0.75), Triplets(subject=Span(start=525, 

In [3]:
from relik import Relik
from transformers import pipeline
import json

# Sample news text
text = """UPS plane crashes after takeoff from Louisville airport. A fireball erupts near airport property after reports of a plane crash at Louisville International Airport, Tuesday, Nov. 4, 2025, in Louisville, Ky.
A UPS cargo plane crashed shortly after takeoff from Louisville Muhammad Ali International Airport on Tuesday evening, federal officials said.
According to the Federal Aviation Administration, UPS Flight 2976 went down around 5:15 p.m. local time after departing the airport. The McDonnell Douglas MD-11 was headed to Daniel K. Inouye International Airport in Honolulu.
Flames and heavy smoke were seen rising from the airfield as fire crews worked to extinguish the blaze.
It’s not yet known whether there were any injuries or fatalities. The cause of the crash remains under investigation.
The FAA said both the agency and the National Transportation Safety Board will investigate."""

# --- Step 1: Entity and relation extraction with Relik ---
relik = Relik.from_pretrained(
    "relik-ie/relik-cie-small",
    device="cuda",
    index_device="cpu",
    precision="fp16",
    topk=50,
)

out = relik(text)

entities = {
    span.text: {"type": span.label, "start": span.start, "end": span.end}
    for span in out.spans
}
relations = [
    {
        "subject": t.subject.text,
        "relation": t.label,
        "object": t.object.text,
        "confidence": t.confidence,
    }
    for t in out.triplets
]

# --- Step 2: Event extraction using LLM ---
# You can replace "google/flan-t5-large" with any text2text LLM of your choice
event_extractor = pipeline(
    "text2text-generation", model="google/flan-t5-large", device=0
)

prompt = f"""
Extract all events from the following text. 
For each event, return JSON with:
- event_type
- trigger_sentence
- participants (entities involved)
- location (if any)
- time (if any)
Text: {text}
"""

llm_output = event_extractor(prompt, max_length=1024)
# The model returns text, we try to parse it as JSON
try:
    events = json.loads(llm_output[0]["generated_text"])
except:
    # fallback: return as a single string if JSON fails
    events = llm_output[0]["generated_text"]

# --- Step 3: Combine entities, relations, and events into KG format ---
kg = {
    "nodes": [{"name": e, "type": entities[e]["type"]} for e in entities],
    "edges": [
        {
            "from": r["subject"],
            "to": r["object"],
            "label": r["relation"],
            "confidence": r["confidence"],
        }
        for r in relations
    ],
    "events": events,
}

# --- Output KG ---
print(json.dumps(kg, indent=2))

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/lightning/fabric/__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2025-12-13 15:45:06,207] [INFO] [relik.inference.annotator.from_pretrained:700] [PID:125025] [RANK:0] Loading Relik from relik-ie/relik-cie-small
[2025-12-13 15:45:06,210] [INFO] [relik.inference.annotator.from_pretrained:701] [PID:125025] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'span': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'relik-ie/index-e5-base-v2-wikipedia-matryoshka',
        },
        'triplet': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'relik-ie/encoder-e5-small-v2-wikipedia-relations-index',
        },
    },
    'metadata_fields': [],
    'reader': {
        '_target_': 'relik.reader.pytorch_modules.triplet.RelikReaderForTripletExtraction',
        'transformer_model': 'relik-ie/relik-reader-deberta-v3-small-cie-wikipedia',
        'use_nme': True,
    },


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

{
  "nodes": [
    {
      "name": "UPS",
      "type": "UPS Airlines"
    },
    {
      "name": "Louisville airport",
      "type": "Louisville International Airport"
    },
    {
      "name": "Louisville International Airport",
      "type": "Louisville International Airport"
    },
    {
      "name": "Nov. 4, 2025",
      "type": "--NME--"
    },
    {
      "name": "4, 2025",
      "type": "--NME--"
    },
    {
      "name": "Louisville, Ky.",
      "type": "Louisville, Kentucky"
    },
    {
      "name": "Louisville Muhammad Ali International Airport",
      "type": "Louisville International Airport"
    },
    {
      "name": "Federal Aviation Administration",
      "type": "Federal Aviation Administration"
    },
    {
      "name": "UPS Flight 2976",
      "type": "--NME--"
    },
    {
      "name": "2976",
      "type": "--NME--"
    },
    {
      "name": "local time",
      "type": "Eastern Time Zone"
    },
    {
      "name": "the airport",
      "type": "Cleveland H